In [1]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6901e609-6140-42ca-8455-fda4abbd82ce;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 104ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/23 00:30:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/23 00:30:34 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/23 00:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
%run trace_encode.ipynb

In [3]:
#first question I have is how many events do I have when I call readfirst
df = readFirstEvents()

12:30AM UTC on Mar 23, 2023 --- read time: 5.271172046661377 seconds ---


In [4]:
#what is our count
df.count()

4595692

In [5]:
#how many days do we have
df.select('event_day').distinct().show()

+---------+
|event_day|
+---------+
|       23|
|       25|
|       24|
+---------+



In [6]:
#how many hours
df.select('event_hour').distinct().show()

+----------+
|event_hour|
+----------+
|        12|
|        22|
|         1|
|        13|
|        16|
|         6|
|         3|
|        20|
|         5|
|        19|
|        15|
|        17|
|         9|
|         4|
|         8|
|        23|
|         7|
|        10|
|        21|
|        11|
+----------+
only showing top 20 rows



In [7]:
#how many malicious events do you have? 
df_m = df.where(col('malicious') == 1)
print("mal count: " +str(df_m.count()))
#how many benign events do you have? 
df_b = df.where(col('malicious') == 0)
print("benign count: " +str(df_b.count()))

mal count: 8596


benign count: 4587096


In [8]:
#compare proportions with original work, orig proportion is .0016%
print('mal_prop = ' + str((8596/4587096)*100))


mal_prop = 0.18739524963070317


In [12]:
#so the prop is higher than the orig. We can live with that, how many mal events are on each day
df_m.groupBy('event_day').count().show()#.select('event_day').agg('count').distinct().count()
#I see these counts as generally in line with what the ANUBIS authors found.

+---------+-----+
|event_day|count|
+---------+-----+
|       23| 3624|
|       25|  459|
|       24| 4513|
+---------+-----+



In [13]:
df_b.groupBy('event_day').count().show()

+---------+-------+
|event_day|  count|
+---------+-------+
|       23|1437108|
|       25|1347693|
|       24|1802295|
+---------+-------+



In [65]:
#function to generate fake data
from pyspark.sql.types import StringType
import random

def gen_dummy_data(out,size):
    start_time = time.time()
    
    if out == 'b':
        # Define a function to generate binary strings with length 25
        def generate_binary_string():
            return format(random.randint(54360,33554432),'b').zfill(25)
    else:
        def generate_binary_string():
            return format(random.randint(1,10000000),'b').zfill(25)
        
    # Create a list of column names
    columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
    
    # Create a list of row data, each containing six binary strings
    data = [[generate_binary_string() for i in range(6)] for j in range(size)]

    # Create a PySpark dataframe from the row data and column names
    df = spark.createDataFrame(data, columns)
    
    #create an array of all the traces, combine in one column, and drop all the other
    #columns
    df = df.select(array('col1', 'col2',
                           'col3','col4', 
                           'col5', 'col6').alias('event_sequence'))
    
    #add the right malicious column
    if out == 'b':
        df = df.withColumn("malicious", lit(0))
        df = df.select("malicious", 'event_sequence')
        
    else: 
        df = df.withColumn("malicious", lit(1))
        df = df.select("malicious", 'event_sequence')
    
    #add a trace column in the front of the df
    df = df.withColumn("Trace", (monotonically_increasing_id() + 1))
    df = df.select("Trace", "malicious", 'event_sequence')
    
    print("elapsed: "+ str(time.time() - start_time))
    
    return df


    

In [59]:
df_samp_b = gen_dummy_data('b',1000)
df_samp_b.limit(10).show()
test = df_samp_b.limit(10).select('event_sequence').collect()
print(len(test[0][0]))
print(len(test[0][0][3]))
df_samp_b.unpersist()

elapsed: 0.07620739936828613
+-----+---------+--------------------+
|Trace|malicious|      event_sequence|
+-----+---------+--------------------+
|    1|        0|[0011110011010001...|
|    2|        0|[0110000100111000...|
|    3|        0|[1011110110110110...|
|    4|        0|[1011100110101010...|
|    5|        0|[0100001100111010...|
|    6|        0|[0011111010100010...|
|    7|        0|[1110100101001111...|
|    8|        0|[1110111011100101...|
|    9|        0|[0010111111110010...|
|   10|        0|[1001101101111000...|
+-----+---------+--------------------+

6
25


DataFrame[Trace: bigint, malicious: int, event_sequence: array<string>]

In [60]:
#now generate all the real data
df_samp_b500k = gen_dummy_data('b',500000)

elapsed: 18.098712921142578


In [61]:
df_samp_brest = gen_dummy_data('b',2091594)

elapsed: 74.34329056739807
elapsed: 0.2696566581726074


In [66]:
df_samp_m = gen_dummy_data('m',6517)
df_samp_m.limit(3).show()

elapsed: 0.26936888694763184
+-----+---------+--------------------+
|Trace|malicious|      event_sequence|
+-----+---------+--------------------+
|    1|        1|[0100000101010000...|
|    2|        1|[0100011100110001...|
|    3|        1|[0010111100010100...|
+-----+---------+--------------------+



In [62]:
#union it all and then write it to parquet
df_samp_b_all = df_samp_b500k.union(df_samp_brest)
#show me some sample rows to be sure
df_samp_b_all.limit(3).show()

+-----+---------+--------------------+
|Trace|malicious|      event_sequence|
+-----+---------+--------------------+
|    1|        0|[1100111110101110...|
|    2|        0|[1110111001110011...|
|    3|        0|[1110000000110100...|
+-----+---------+--------------------+

+-----+---------+--------------------+
|Trace|malicious|      event_sequence|
+-----+---------+--------------------+
|    1|        1|[0000000001110101...|
|    2|        1|[0000000000111100...|
|    3|        1|[0000000000010001...|
+-----+---------+--------------------+



In [67]:
df_samp_all = df_samp_b_all.union(df_samp_m)

In [69]:
#now write the first set to parquet as a train test dataset
df_samp_all.write.option("maxRecordsPerFile", 300000).mode("overwrite")\
.parquet(f'{s3_url_trusted}/prod/tensor_sample_data/train_val')

In [70]:
#now we do this all over again for the holdout set
df_b = gen_dummy_data('b',1347693)
df_m = gen_dummy_data('m', 459)
df_all = df_b.union(df_m)
df_samp_all.write.option("maxRecordsPerFile", 300000).mode("overwrite")\
.parquet(f'{s3_url_trusted}/prod/tensor_sample_data/test_holdout')

elapsed: 47.36942958831787
elapsed: 0.053705453872680664


In [9]:
#does first events work now on data duplication
cg = create_graph(df_m)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 1261


In [10]:
cg.limit(10).toPandas()
#answer I belive it did. 

,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, 36d062d...","(36d062d4-1f69-4dcc-9791-46b5c07dadef,)"
1,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4b9755a...","(4b9755ad-05bb-458e-b9ff-27852eb51860,)"
2,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 50ac366...","(50ac366a-4500-4ef0-8579-22adacaf6438,)"
3,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 6fba2e2...","(6fba2e2c-1552-41ad-80ee-fffdfbe58d29,)"
4,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 75c93ca...","(75c93ca7-955f-4e24-b728-cb5dd9ed1d2e,)"
5,"(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, 7f91e81...","(7f91e814-3f77-4ee9-bfee-01d796b8f424,)"
6,"(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245